In [2]:
#LangChain
import os
from dotenv import load_dotenv
%pip install kaggle --quiet
import kaggle
# Cargar variables de entorno desde .env
load_dotenv()
os.environ["KAGGLE_USERNAME"] = os.getenv("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = os.getenv("KAGGLE_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# Descargar el dataset (si no existe)
dataset = "rajneesh231/lex-fridman-podcast-transcript"
path = " .. /data/13langchain"
kaggle.api.dataset_download_files(dataset, path=path, unzip=True)


Note: you may need to restart the kernel to use updated packages.


OSError: Could not find kaggle.json. Make sure it's located in C:\Users\jperugachi\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [ ]:
from langchain.document_loaders import CSVLoader
import pandas as pd
# Cargar el dataset CSV
file_path = os.path. join(path, "podcastdata_dataset.csv") # ajusta nombre si difiere
df = pd.read_csv(file_path)
df

In [ ]:
from langchain. schema import Document
# Convertir cada fila en un Document
documents = [
Document(
page_content=row["text"],
metadata={"id": row["id"], "guest": row["guest"], "title": row["title"]},
   )
    for _, row in df.iterrows()
]    
documents

In [ ]:
from langchain. text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [chunk.page_content for chunk in chunks]
vectorstore = FAISS.from_texts(texts, embeddings)
vectorstore.save_local("index_13langchain")

Indexacion FAISS
